In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/meta-dataset-20/meta-dataset-20.csv
/kaggle/input/meta-dataset-20/meta-dataset-21.csv
/kaggle/input/darbar-audio/darbar_audio/RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp.mp3
/kaggle/input/darbar-audio/darbar_audio/RaagMalkaunsIndraniMukherjeeKhayalVocalMusicofIndiamp.mp3
/kaggle/input/darbar-audio/darbar_audio/SunriseRaagBhairaviManjushaPatilIndianClassicalMusicVRmp.mp3
/kaggle/input/darbar-audio/darbar_audio/MilindMalsheRaagHindolPanchammp.mp3
/kaggle/input/darbar-audio/darbar_audio/SublimeRaagBhimpalasiPartTwoPanditKushalDasSatyajitTalwalkarSitarMusicofIndiamp.mp3
/kaggle/input/darbar-audio/darbar_audio/MilindMalsheRaagJogiyamp.mp3
/kaggle/input/darbar-audio/darbar_audio/MorningRaagParameshwariPtRonuMajumdarSukhvinderSinghBansuriTablaMusicofIndiamp.mp3
/kaggle/input/darbar-audio/darbar_audio/RaagGawatiManjiriAsanareKelkarAfternoonRagasDarbarVRmp.mp3
/kaggle/input/darbar-audio/darbar_audio/RareIndianInstrumenttheEsrajRaagBhairaviArshadKhanMusicofIndiamp

In [7]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn import hmm
from joblib import dump

# Directory containing the audio files
dir_path = '/kaggle/input/darbar-audio/darbar_audio'

# Function to extract MFCC features from an audio segment
def extract_segment_mfcc(segment, sr):
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=5)  # Extract MFCC features
    return mfccs.T  # Transpose the MFCC matrix to have features along columns

# Function to normalize MFCC features using z-score normalization
def normalize_mfccs(mfccs_array):
    scaler = StandardScaler()
    normalized_mfccs = [scaler.fit_transform(mfccs) for mfccs in mfccs_array]
    return normalized_mfccs

# Function to build HMM model for given MFCC features
def build_hmm_model(mfcc_features):
    model = hmm.GaussianHMM(n_components=5, covariance_type="diag", n_iter=100)
    model.fit(mfcc_features)
    return model

# Function to calculate similarity between HMM models
def calculate_similarity(hmm_models, mfccs_list):
    n_songs = len(hmm_models)
    similarities = np.zeros((n_songs, n_songs))
    likelihoods = []

    # Calculate log likelihoods for each HMM model
    for hmm_model in hmm_models:
        hmm_likelihoods = []
        for mfccs in mfccs_list:
            log_likelihood = hmm_model.score(mfccs)  # Calculate the log likelihood
            hmm_likelihoods.append(log_likelihood)
        likelihoods.append(hmm_likelihoods)

    # Calculate similarity between HMM models
    for i in range(n_songs):
        for j in range(n_songs):
            Ni = len(likelihoods[i])
            Nj = len(likelihoods[j])
            sim = ((likelihoods[i][i]-likelihoods[i][j]))/Ni+((likelihoods[j][j]-likelihoods[j][i]))/Nj
            if sim==0:
                similarities[i, j] = sim
                continue
            similarities[i, j] = np.log(sim)

    return similarities

# Main function
def main():
    # List to store HMM models for each song
    hmm_models = []
    mfccs_list = []
    file_names=[]
    cnt=0
    # Iterate over each audio file in the directory
    for filename in os.listdir(dir_path):
#         if cnt==3:
#             break
        if filename.endswith('.mp3'):  # Assuming all audio files are in mp3 format
            file_path = os.path.join(dir_path, filename)
            file_name, file_extension = os.path.splitext(os.path.basename(file_path))
            print(file_name) 
            file_names.append(file_name)
            audio_path = os.path.join(dir_path, filename)
            y, sr = librosa.load(audio_path)  # Load audio file
            segment_length = 2000  # Length of each segment in milliseconds
            n_samples_per_segment = int((segment_length / 1000) * sr)
            n_segments = len(y) // n_samples_per_segment

            # Extract MFCC features for each segment
            mfccs = [extract_segment_mfcc(y[i * n_samples_per_segment:(i + 1) * n_samples_per_segment], sr)
                          for i in range(n_segments)]

            # Normalize the MFCC features for all segments

            # Concatenate normalized MFCC features for all segments
            concatenated_mfccs = np.concatenate(mfccs)
            mfccs_list.append(concatenated_mfccs)

            # Build and store HMM model for the song
#             hmm_model = build_hmm_model(concatenated_mfccs)
#             hmm_models.append(hmm_model)
        cnt+=1
    
    normalized_mfccs = normalize_mfccs(mfccs_list)
    print(file_names)
    for m in range(len( mfccs_list)):
        # Build and store HMM model for the song
        hmm_model = build_hmm_model( mfccs_list[m])
        hmm_models.append(hmm_model)
        dump(hmm_model, f'{file_names[m]}.pkl')# 
    

# #     Calculate similarity between HMM models
#     similarities = calculate_similarity(hmm_models,  mfccs_list)
#     print("Similarity matrix:")
#     print(similarities)
#     return similarities;

# Execute the main function
if __name__ == "__main__":
    main()


RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp



KeyboardInterrupt



In [8]:

from joblib import dump
cnt=1
for i, hmm_model in enumerate(hmm_models):
        # Save each HMM model to a pickle file
    dump(hmm_model, f'{cnt}.pkl')# 
    cnt+=1

In [123]:
from joblib import load

hmm_models = []
mfccs_list = []

    # Load HMM models from pickle files
for filename in os.listdir('/kaggle/working/'):
    if len(hmm_models) == 3:  # Limiting to 3 audio files for demonstration
        break
    if filename.endswith('.pkl'):  # Assuming all pickle files contain HMM models
        print(filename)
        hmm_model = load(os.path.join('/kaggle/working/', filename))
        hmm_models.append(hmm_model)
        
hmm_models

1.pkl
2.pkl
3.pkl


[GaussianHMM(n_components=8, n_iter=100),
 GaussianHMM(n_components=8, n_iter=100),
 GaussianHMM(n_components=8, n_iter=100)]

In [131]:
hmm_models[0].score(concatenated_mfcc[1])

-270813.95502032014

In [155]:
!ls

 AMajesticalMeetingJesseBannisterandSukhwinderSinghRaagVachaspatimp.pkl
 AbhisekLahiriSukhvinderSinghPinkyRaagChayanatSarodTablamp.pkl
 AbhisekLahiriSukhvinderSinghPinkyRaagShreeSarodTablamp.pkl
 AjoyChakrabartyAahirBhaironmp.pkl
 AjoyChakrabartyBairagimp.pkl
 AjoyChakrabartyBilaskhaniTodimp.pkl
 AjoyChakrabartyIntroductionaboutAjoyChakrabartymp.pkl
 AjoyChakrabartyIntroductoryspeechbyAjoyChakrabartymp.pkl
 AjoyChakrabartyKalavatimp.pkl
 AjoyChakrabartyLyricsofalltheBandishesmp.pkl
 AjoyChakrabartyMalkaunsmp.pkl
 AjoyChakrabartyMaruBihagmp.pkl
 AjoyChakrabartyMultanimp.pkl
 AjoyChakrabartyNatBhaironmp.pkl
 AjoyChakrabartyRaagAbhogimp.pkl
 AjoyChakrabartyRaagJogmp.pkl
 AjoyChakrabartyRaagKedarmp.pkl
 AjoyChakrabartyRaagKhamajmp.pkl
 AjoyChakrabartyRaagRageshrimp.pkl
 AjoyChakrabartyRaageshreemp.pkl
 AjoyChakrabartyRagaLalitKhayalmp.pkl
 AjoyChakrabartyRagaShreeKhayalmp.pkl
 AjoyChakrabartyShrutinandanConceptanIntroductionbyPanditAjoyChakrabartymp.pkl
 AjoyChakrabartySudhKalyanmp.pkl
 Aj

In [157]:
!zip -r file.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/BeautifulMorningRaagBhatiyarMitaNagSoloSitarMusicofIndiamp.pkl (deflated 22%)
  adding: kaggle/working/EndearingRaagPilooSahanaBanerjeeSoloSitarMusicofIndiamp.pkl (deflated 25%)
  adding: kaggle/working/RaagKaunsiKanadaSoumikDattaGurdainRayattSarodTablaMusicofIndiamp.pkl (deflated 25%)
  adding: kaggle/working/MesmerisingRaagLalitIndraniMukherjeeKiranaRampurKhayalMusicofIndiamp.pkl (deflated 27%)
  adding: kaggle/working/RaagKomalRishabhAsavariPelvaNaikMorningDhrupadDarbarVRmp.pkl (deflated 23%)
  adding: kaggle/working/RaagMultaniManjushaPatilAfternoonDarbarVRmp.pkl (deflated 22%)
  adding: kaggle/working/RaagGawatiManjiriAsanareKelkarAfternoonRagasDarbarVRmp.pkl (deflated 24%)
  adding: kaggle/working/SatyasheelDeshpandeRaagChandrakaunsmp.pkl (deflated 22%)
  adding: kaggle/working/RareRaagAhiriHarmeetVirdeeBhupinderChaggarSitarTablaMusicofIndiamp.pkl (deflated 25%)
  adding: kaggle/working/RaagMadhuvantiArshadKhanMusicof

In [160]:

hmm_models


[GaussianHMM(n_components=8, n_iter=100),
 GaussianHMM(n_components=8, n_iter=100),
 GaussianHMM(n_components=8, n_iter=100)]

In [8]:
import numpy as np
l1 = [[[0,1],[0,2],[1,1]], [[0,1],[0,3],[4,4]]]
l2 = np.concatenate(l1)
l2

array([[0, 1],
       [0, 2],
       [1, 1],
       [0, 1],
       [0, 3],
       [4, 4]])

In [9]:
l1=np.asarray(l1)
l1[0]

array([[0, 1],
       [0, 2],
       [1, 1]])

In [3]:
!pip install --upgrade --user hmmlearn

In [2]:
import os
import numpy as np
import librosa
from hmmlearn import hmm
import pickle
from joblib import load

# Directory paths
song_dir = '/kaggle/input/darbar-audio/darbar_audio'
model_dir = '/kaggle/input/hmm-audio/models'
output_dir = '/kaggle/working/'

# Function to extract MFCC features from an audio segment
def extract_segment_mfcc(segment, sr):
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=5)  # Extract MFCC features
    return mfccs.T  # Transpose the MFCC matrix to have features along columns

# Load HMM models
hmm_models = {}
for model_file in os.listdir(model_dir):
    if model_file.endswith('.pkl'):
        model_name = os.path.splitext(model_file)[0]
#         print(model_name)
        hmm_model = load(os.path.join(model_dir, model_file))
        hmm_models[model_name] = hmm_model

cnt=0
# Iterate over each song
scores = {}

for song_file in os.listdir(song_dir):
    if song_file.endswith('.mp3'):
        song_name = os.path.splitext(song_file)[0]
        song_path = os.path.join(song_dir, song_file)
        
        # Load song
        y, sr = librosa.load(song_path)

        # Extract MFCC features for each segment
        segment_length = 2000  # Length of each segment in milliseconds
        n_samples_per_segment = int((segment_length / 1000) * sr)
        n_segments = len(y) // n_samples_per_segment
        mfccs_list = [extract_segment_mfcc(y[i * n_samples_per_segment:(i + 1) * n_samples_per_segment], sr)
                      for i in range(n_segments)]
        mfccs_list=np.concatenate(mfccs_list)
        # Calculate scores for each HMM model
        for model_name, hmm_model in hmm_models.items():
           
            try:
                log_likelihood = hmm_model.score(mfccs_list)
#                 print(model_name, song_name)
                scores[(model_name, song_name)] = log_likelihood
            except Exception as e:
                cnt+=1
                print(f"{cnt} Error processing {model_name} with {song_name}: {e}")
                continue

# Save scores to numpy file
output_file = os.path.join(output_dir, 'scorses.npy')
# np.save(output_file, scores)


1 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
2 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with RaagMalkaunsIndraniMukherjeeKhayalVocalMusicofIndiamp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
3 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with SunriseRaagBhairaviManjushaPatilIndianClassicalMusicVRmp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
4 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with MilindMalsheRaagHindolPanchammp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
5 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with SublimeRaagBhimpalasiPartTwoPanditKushalDasSatyajitTalwalkarSitarMusicofIndiamp: transmat_ rows must sum to 1 (got row sums of [1.

Note: Illegal Audio-MPEG-Header 0x44d3a352 at offset 54923548.
Note: Trying to resync...
Note: Skipped 244 bytes in input.


107 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with OmkarDadarkarRaagAhirBhaironmp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
108 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with TranceDhrupadUstadBahauddinDagarRaagKausiKanadaMusicofIndiamp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
109 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with RaagGorakhKalyanSoumikDattaSukhvinderSinghPinkySarodTablaMusicofIndiamp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
110 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with RamakantGaikwadRaagMadhuvantiKiranaandPatialagharanasmp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
111 Error processing PanditUlhasKashalkarRaagDarbariKhayalVocalMusicofIndiamp with OmkarDadarkarRaagGavtimp: transmat_ rows must sum to 1 (got row sums of [1. 0. 1. 1. 1.])
112 Error proc

import logging
import string
import sys
from collections import deque

import numpy as np
from scipy import linalg, special
from sklearn.base import BaseEstimator
from sklearn.utils.validation import (
    check_array, check_is_fitted, check_random_state)

from . import _hmmc, _kl_divergence as _kl, _utils
from .utils import normalize, log_normalize


_log = logging.getLogger(__name__)
#: Supported decoder algorithms.
DECODER_ALGORITHMS = frozenset(("viterbi", "map"))


class ConvergenceMonitor:
    """
    Monitor and report convergence to :data:`sys.stderr`.

    Attributes
    ----------
    history : deque
        The log probability of the data for the last two training
        iterations. If the values are not strictly increasing, the
        model did not converge.
    iter : int
        Number of iterations performed while training the model.

    Examples
    --------
    Use custom convergence criteria by subclassing ``ConvergenceMonitor``
    and redefining the ``converged``

In [3]:
count = 0
for key, value in scores.items():
    print(key, "->", value)
    count += 1
    if count == 5:
        break

('PravinGodkhindiRaagYamanBansuriatRavennaFestivalMusicofIndiamp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -593319.7746263527
('BestMorningMeditationRaagBasantMukhariKaushikiChakrabortyDarbarVRmp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -919558.9079218834
('AjoyChakrabartyAahirBhaironmp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -606299.1340488682
('RoyalRaagDarbariJorJhallaIrshadKhanSurbaharMusicofIndiamp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -589866.3373786754
('SpellbindingRaagMalkaunsAbhisekLahiriMusicofIndiamp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -698991.736106542


In [5]:
scores_str_keys = {str(key): value for key, value in scores.items()}

# Define the file path
file_path = "/kaggle/working/scores.json"

# Write the dictionary with string keys to a JSON file
with open(file_path, "w") as json_file:
    json.dump(scores_str_keys, json_file)

print("Scores dictionary has been successfully stored in 'scores.json' file.")

Scores dictionary has been successfully stored in 'scores.json' file.


In [8]:
import json

# Define the file path
file_path = "/kaggle/working/scores.json"

# Load the JSON file into a dictionary
with open(file_path, "r") as json_file:
    loaded_scores = json.load(json_file)

print("Dictionary loaded from JSON file:")
count = 0
for key, value in loaded_scores.items():
    print(key, "->", value)
    count += 1
    if count == 5:
        break

Dictionary loaded from JSON file:
('PravinGodkhindiRaagYamanBansuriatRavennaFestivalMusicofIndiamp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -593319.7746263527
('BestMorningMeditationRaagBasantMukhariKaushikiChakrabortyDarbarVRmp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -919558.9079218834
('AjoyChakrabartyAahirBhaironmp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -606299.1340488682
('RoyalRaagDarbariJorJhallaIrshadKhanSurbaharMusicofIndiamp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -589866.3373786754
('SpellbindingRaagMalkaunsAbhisekLahiriMusicofIndiamp', 'RaamSimaarEhaTeroKaajBhaiBaldeepSinghRaagJaijaiwantiGurbanimp') -> -698991.736106542


In [15]:
l=[]
m=[]
for song_file in os.listdir('/kaggle/input/darbar-audio/darbar_audio'):
    l.append(song_file)
    
for model_file in os.listdir(model_dir):
    
    m.append(model_file)
    

In [18]:
def find_matching_index(l, m):
    min_len = min(len(l), len(m))
    matching_index = 0
    for i in range(min_len):
        if l[i] == m[i]:
            matching_index = i + 1
        else:
            break
    return matching_index
matching_index = find_matching_index(l, m)
print("Matching index:", matching_index)
m[0]

Matching index: 0


'PravinGodkhindiRaagYamanBansuriatRavennaFestivalMusicofIndiamp.pkl'